In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 9.6 MB/s 
     |████████████████████████████████| 120 kB 72.5 MB/s 
     |████████████████████████████████| 6.6 MB 48.2 MB/s 


In [ ]:
pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 20.8 MB/s 


In [ ]:
pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 707 kB 33.9 MB/s 
     |████████████████████████████████| 5.9 MB 54.9 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.


# Importing Libaraires


In [ ]:

# Add environment Packages paths to conda
import os, sys, warnings
import pandas as pd
import numpy as np
warnings.simplefilter("ignore")

# Text preprocessing packages
import nltk # Text libarary
# nltk.download('stopwords')
import string # Removing special characters {#, @, ...}

# Modelling
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score
from sklearn.svm import SVC
# Saving Model
import pickle

# Visualization Packages
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.3)
%matplotlib inline


from pylab import rcParams
import joblib

from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn,optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics.functional import f1_score, accuracy
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report

from tqdm.auto import tqdm

torch.cuda.empty_cache()


In [ ]:
torch.cuda.empty_cache()

In [ ]:
#import the dataset
path_input = '/content/cleaned_data_ArabicLightStemmer.csv'
df = pd.read_csv(path_input)
df.dropna(inplace=True)
df.head()

,text,label
0,الاوليمبياد جايه هك سه كليه,none
1,عجز موازنه صل ل ناتج محل عني سه قل فلس هايم سه...,anger
2,تنا يل حظ هباب ضح,sadness
3,جميع ريد حقيق هداف ونس تالق حراس مرم,joy
4,الاوليمبياد ظام مختلف مواعيد مونديال مكانتش مق...,none


In [ ]:
#print the shape of the dataset
df.shape

(9673, 2)

In [ ]:
#check for null values
df.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
#check for duplication
df.duplicated().sum()

0

In [ ]:
#make train and test data and save it as a csv file
train, val = train_test_split(df[['label','text']], test_size=0.2)


lbl_enc = LabelEncoder()
train.loc[:,"label"] = lbl_enc.fit_transform(train["label"])
val.loc[:,"label"] = lbl_enc.transform(val["label"])

joblib.dump(lbl_enc,"label_encoder.pkl")

train.to_csv("train.csv",index=False)
val.to_csv("test.csv",index=False)

In [ ]:
#print the classes and it's encoding
lbl_enc.classes_
{v: k for v, k in enumerate(lbl_enc.classes_)}

{0: 'anger',
 1: 'fear',
 2: 'joy',
 3: 'love',
 4: 'none',
 5: 'sadness',
 6: 'surprise',
 7: 'sympathy'}

In [ ]:
#make a new data to test different examples in our data
new_data = df.iloc[:2]
new_data

new_text  = {'text': 'حب','label': 'love'}
new_data = new_data.append(new_text, ignore_index = True)
new_text  = {'text': 'مستغرب','label': 'surprise'}
new_data = new_data.append(new_text, ignore_index = True)
new_text  = {'text': 'حزن','label': 'sadness'}
new_data = new_data.append(new_text, ignore_index = True)
new_data.loc[:,"label"] = lbl_enc.transform(new_data["label"])
new_data.to_csv("new_data.csv",index=False)

new_data


,text,label
0,الاوليمبياد جايه هك سه كليه,4
1,عجز موازنه صل ل ناتج محل عني سه قل فلس هايم سه...,0
2,حب,3
3,مستغرب,6
4,حزن,5


In [ ]:
class ArabicDataset(Dataset):
    def __init__(self,data,max_len,model_type="Twitter"):
        super().__init__()
        self.labels = data["label"].values
        self.texts = data["text"].values
        self.max_len = max_len
        model = {"Twitter": "aubmindlab/bert-base-arabertv02-twitter",
                "ARBERT": "UBC-NLP/ARBERT",
                "Base": "aubmindlab/bert-base-arabertv02",
                "MARBERT": "UBC-NLP/MARBERT"}
        self.tokenizer = AutoTokenizer.from_pretrained(model[model_type])
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self,idx):
        text = " ".join(self.texts[idx].split())
        label = self.labels[idx]
        inputs = self.tokenizer(text,padding='max_length',
                                max_length=self.max_len,truncation=True,return_tensors="pt")
        
        #input_ids,token_type_ids,attention_mask
        return {
            "inputs":{"input_ids":inputs["input_ids"][0],
                      "token_type_ids":inputs["token_type_ids"][0],
                      "attention_mask":inputs["attention_mask"][0],
                     },
            "labels": torch.tensor(label,dtype=torch.long) 
        }

In [ ]:
class ArabicDataModule(pl.LightningDataModule):
    def __init__(self,train_path,val_path,new_data_path,batch_size=12,max_len=100,model_type="Twitter"):
        super().__init__()
        self.train_path,self.val_path,self.new_data_path= train_path,val_path,new_data_path
        self.batch_size = batch_size
        self.max_len = max_len
        self.model_type = model_type
    
    def setup(self,stage=None):
        train = pd.read_csv(self.train_path)
        val = pd.read_csv(self.val_path)
        new_data = pd.read_csv(self.new_data_path)
        self.train_dataset = ArabicDataset(data=train,max_len=self.max_len,model_type=self.model_type)
        self.val_dataset = ArabicDataset(data=val,max_len=self.max_len,model_type=self.model_type)
        self.new_dataset = ArabicDataset(data=new_data,max_len=self.max_len,model_type=self.model_type)

    def train_dataloader(self):
        return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.new_dataset,batch_size=self.batch_size,shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)

In [ ]:
n_classes = 8
class ArabicBertModel(pl.LightningModule):
    def __init__(self,model_type="Twitter"):
        super().__init__()
        model = {"Twitter": ("aubmindlab/bert-base-arabertv02-twitter",768),
                "ARBERT": ("UBC-NLP/ARBERT",768),
                 "Base": ("aubmindlab/bert-base-arabertv02",768),
                "MARBERT": ("UBC-NLP/MARBERT",768)}
        self.bert_model = AutoModel.from_pretrained(model[model_type][0])
        self.fc = nn.Linear(model[model_type][1],n_classes)
    
    def forward(self,inputs):
        out = self.bert_model(**inputs)#inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])
        last_hidden_states = out[1]
        out = self.fc(last_hidden_states)
        return out
    
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=1e-6)
    
    def criterion(self,output,target):
        return nn.CrossEntropyLoss()(output,target)
    
    #TODO: adding metrics
    def training_step(self,batch,batch_idx):
        x,y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_result = f1_score(out, y, num_classes=n_classes)
        metrics = {"train_f1": f1_result, "train_loss": loss}
        self.log_dict(metrics)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x, y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_result = f1_score(out, y, num_classes=n_classes)
        metrics = {"val_f1": f1_result, "val_loss": loss}
        self.log_dict(metrics)
        return metrics

# Training On MARBERT

## LR = 1e-6

#### 10 epoch

In [ ]:
# with lr = 1e-6 
MODEL_TYPE = "MARBERT"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=128, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=10, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params
-----------------------------------------
0 | bert_model | BertModel | 162 M 
1 | fc         | Linear    | 6.2 K 
-----------------------------------------
162 M     Trainable params
0         Non-trainable params
162 M     Total params
651.390   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


##### Training Results

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

  0%|          | 0/16 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       anger       0.64      0.76      0.70       288
        fear       0.93      0.84      0.88       245
         joy       0.53      0.29      0.37       251
        love       0.70      0.72      0.71       215
        none       0.57      0.96      0.72       307
     sadness       0.61      0.38      0.47       241
    surprise       0.46      0.30      0.37       198
    sympathy       0.73      0.84      0.78       190

    accuracy                           0.65      1935
   macro avg       0.65      0.64      0.62      1935
weighted avg       0.65      0.65      0.63      1935



In [ ]:
results={}
Overall_Results={}
sum_f1=0
sum_precision=0
sum_recall=0
for i in lbl_enc.classes_:
  sum_f1 += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['f1-score']
  sum_precision += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['precision']
  sum_recall += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['recall']

results['Accuracy'] =round(classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)['accuracy'],2)
results['Average_F1'] =  round(sum_f1 /8,2)
results['Average_Precision'] =  round(sum_precision /8,2)
results['Average_Recall'] =  round(sum_recall /8,2)



Overall_Results['(10_epoch)_(1e-6_LR)_(128_batch)'] = results

print(results)
print(Overall_Results)

{'Accuracy': 0.65, 'Average_F1': 0.62, 'Average_Precision': 0.65, 'Average_Recall': 0.64}
{'(10_epoch)_(1e-6_LR)_(128_batch)': {'Accuracy': 0.65, 'Average_F1': 0.62, 'Average_Precision': 0.65, 'Average_Recall': 0.64}}


#### 20 epoch

In [ ]:
# with lr = 1e-6 
MODEL_TYPE = "MARBERT"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=128, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


##### Training Results

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

  0%|          | 0/16 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       anger       0.68      0.73      0.70       288
        fear       0.93      0.86      0.90       245
         joy       0.57      0.41      0.48       251
        love       0.72      0.72      0.72       215
        none       0.62      0.92      0.74       307
     sadness       0.59      0.47      0.53       241
    surprise       0.47      0.37      0.42       198
    sympathy       0.80      0.84      0.82       190

    accuracy                           0.68      1935
   macro avg       0.67      0.67      0.66      1935
weighted avg       0.67      0.68      0.67      1935



In [ ]:
results={}
sum_f1=0
sum_precision=0
sum_recall=0
for i in lbl_enc.classes_:
  sum_f1 += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['f1-score']
  sum_precision += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['precision']
  sum_recall += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['recall']

results['Accuracy'] =round(classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)['accuracy'],2)
results['Average_F1'] =  round(sum_f1 /8,2)
results['Average_Precision'] =  round(sum_precision /8,2)
results['Average_Recall'] =  round(sum_recall /8,2)



Overall_Results['(20_epoch)_(1e-6_LR)_(128_batch)'] = results
print(results)
print(Overall_Results)

{'Accuracy': 0.68, 'Average_F1': 0.66, 'Average_Precision': 0.67, 'Average_Recall': 0.67}
{'(10_epoch)_(1e-6_LR)_(128_batch)': {'Accuracy': 0.65, 'Average_F1': 0.62, 'Average_Precision': 0.65, 'Average_Recall': 0.64}, '(20_epoch)_(1e-6_LR)_(128_batch)': {'Accuracy': 0.68, 'Average_F1': 0.66, 'Average_Precision': 0.67, 'Average_Recall': 0.67}}


#### 30 epoch , batch size = 50 

In [ ]:
MODEL_TYPE = "MARBERT"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=50, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=30, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

##### Training Results

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

  0%|          | 0/39 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       anger       0.69      0.68      0.68       288
        fear       0.94      0.86      0.90       245
         joy       0.55      0.49      0.52       251
        love       0.70      0.73      0.72       215
        none       0.64      0.83      0.73       307
     sadness       0.54      0.54      0.54       241
    surprise       0.48      0.37      0.42       198
    sympathy       0.82      0.80      0.81       190

    accuracy                           0.67      1935
   macro avg       0.67      0.66      0.66      1935
weighted avg       0.67      0.67      0.67      1935



In [ ]:
results={}
sum_f1=0
sum_precision=0
sum_recall=0
for i in lbl_enc.classes_:
  sum_f1 += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['f1-score']
  sum_precision += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['precision']
  sum_recall += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['recall']

results['Accuracy'] =round(classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)['accuracy'],2)
results['Average_F1'] =  round(sum_f1 /8,2)
results['Average_Precision'] =  round(sum_precision /8,2)
results['Average_Recall'] =  round(sum_recall /8,2)


Overall_Results['(30_epoch)_(1e-6_LR)_(50_batch)'] = results
print(results)
Overall_Results

{'Accuracy': 0.67, 'Average_F1': 0.66, 'Average_Precision': 0.67, 'Average_Recall': 0.66}


{'(10_epoch)_(1e-6_LR)_(128_batch)': {'Accuracy': 0.65,
  'Average_F1': 0.62,
  'Average_Precision': 0.65,
  'Average_Recall': 0.64},
 '(20_epoch)_(1e-6_LR)_(128_batch)': {'Accuracy': 0.68,
  'Average_F1': 0.66,
  'Average_Precision': 0.67,
  'Average_Recall': 0.67},
 '(30_epoch)_(1e-6_LR)_(50_batch)': {'Accuracy': 0.67,
  'Average_F1': 0.66,
  'Average_Precision': 0.67,
  'Average_Recall': 0.66}}

#### 30 epoch , batch size = 128 

In [ ]:
MODEL_TYPE = "MARBERT"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=128, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=30, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params
-----------------------------------------
0 | bert_model | BertModel | 162 M 
1 | fc         | Linear    | 6.2 K 
-----------------------------------------
162 M     Trainable params
0         Non-trainable params
162 M     Total params
651.390   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

##### Training Results

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = torch.load('../input/fine-tuning-arabert/arabert_arabic_dialect.pth',  map_location=device)
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))

  0%|          | 0/16 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       anger       0.70      0.70      0.70       275
        fear       0.97      0.92      0.94       222
         joy       0.51      0.54      0.52       216
        love       0.75      0.74      0.75       238
        none       0.69      0.82      0.75       316
     sadness       0.55      0.57      0.56       237
    surprise       0.56      0.38      0.45       219
    sympathy       0.82      0.82      0.82       212

    accuracy                           0.69      1935
   macro avg       0.69      0.69      0.69      1935
weighted avg       0.69      0.69      0.69      1935



In [ ]:
results={}
sum_f1=0
sum_precision=0
sum_recall=0
for i in lbl_enc.classes_:
  sum_f1 += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['f1-score']
  sum_precision += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['precision']
  sum_recall += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['recall']

results['Accuracy'] =round(classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)['accuracy'],2)
results['Average_F1'] =  round(sum_f1 /8,2)
results['Average_Precision'] =  round(sum_precision /8,2)
results['Average_Recall'] =  round(sum_recall /8,2)


Overall_Results['(30_epoch)_(1e-6_LR)_(128_batch)'] = results
print(results)
Overall_Results

{'Accuracy': 0.69, 'Average_F1': 0.69, 'Average_Precision': 0.69, 'Average_Recall': 0.69}


{'(10_epoch)_(1e-6_LR)_(128_batch)': {'Accuracy': 0.65,
  'Average_F1': 0.62,
  'Average_Precision': 0.65,
  'Average_Recall': 0.64},
 '(20_epoch)_(1e-6_LR)_(128_batch)': {'Accuracy': 0.68,
  'Average_F1': 0.66,
  'Average_Precision': 0.67,
  'Average_Recall': 0.67},
 '(30_epoch)_(1e-6_LR)_(50_batch)': {'Accuracy': 0.67,
  'Average_F1': 0.66,
  'Average_Precision': 0.67,
  'Average_Recall': 0.66},
 '(30_epoch)_(1e-6_LR)_(128_batch)': {'Accuracy': 0.69,
  'Average_F1': 0.69,
  'Average_Precision': 0.69,
  'Average_Recall': 0.69}}

In [ ]:
tuning_result = pd.DataFrame(Overall_Results)

In [ ]:
tuning_result

,(10_epoch)_(1e-6_LR)_(128_batch),(20_epoch)_(1e-6_LR)_(128_batch),(30_epoch)_(1e-6_LR)_(50_batch),(30_epoch)_(1e-6_LR)_(128_batch)
Accuracy,0.65,0.68,0.67,0.69
Average_F1,0.62,0.66,0.66,0.69
Average_Precision,0.65,0.67,0.67,0.69
Average_Recall,0.64,0.67,0.66,0.69


## LR = 2e-6

In [ ]:
n_classes = 8
class ArabicBertModel(pl.LightningModule):
    def __init__(self,model_type="Twitter"):
        super().__init__()
        model = {"Twitter": ("aubmindlab/bert-base-arabertv02-twitter",768),
                "ARBERT": ("UBC-NLP/ARBERT",768),
                 "Base": ("aubmindlab/bert-base-arabertv02",768),
                "MARBERT": ("UBC-NLP/MARBERT",768)}
        self.bert_model = AutoModel.from_pretrained(model[model_type][0])
        self.fc = nn.Linear(model[model_type][1],n_classes)
    
    def forward(self,inputs):
        out = self.bert_model(**inputs)#inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])
        last_hidden_states = out[1]
        out = self.fc(last_hidden_states)
        return out
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=2e-06)
    
    def criterion(self,output,target):
        return nn.CrossEntropyLoss()(output,target)
    
    #TODO: adding metrics
    def training_step(self,batch,batch_idx):
        x,y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_result = f1_score(out, y, num_classes=n_classes)
        metrics = {"train_f1": f1_result, "train_loss": loss}
        self.log_dict(metrics)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x, y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_result = f1_score(out, y, num_classes=n_classes)
        metrics = {"val_f1": f1_result, "val_loss": loss}
        self.log_dict(metrics)
        return metrics

#### 10 epoch

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "MARBERT"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=128, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=10, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type      | Params
-----------------------------------------
0 | bert_model | BertModel | 162 M 
1 | fc         | Linear    | 6.2 K 
-----------------------------------------
162 M     Trainable params
0         Non-trainable params
162 M     Total params
651.390   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


##### Training Results

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))



  0%|          | 0/16 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       anger       0.72      0.69      0.71       290
        fear       0.92      0.89      0.90       219
         joy       0.62      0.50      0.55       241
        love       0.78      0.79      0.79       227
        none       0.64      0.92      0.75       316
     sadness       0.61      0.45      0.52       256
    surprise       0.42      0.40      0.41       167
    sympathy       0.85      0.83      0.84       219

    accuracy                           0.70      1935
   macro avg       0.70      0.68      0.68      1935
weighted avg       0.70      0.70      0.69      1935



In [ ]:
results={}
Overall_Results = {}
sum_f1=0
sum_precision=0
sum_recall=0
for i in lbl_enc.classes_:
  sum_f1 += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['f1-score']
  sum_precision += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['precision']
  sum_recall += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['recall']

results['Accuracy'] =round(classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)['accuracy'],2)
results['Average_F1'] =  round(sum_f1 /8,2)
results['Average_Precision'] =  round(sum_precision /8,2)
results['Average_Recall'] =  round(sum_recall /8,2)

Overall_Results['(10_epoch)_(2e-6_LR)_(128_batch)'] = results
print(results)
print(Overall_Results)

{'Accuracy': 0.7, 'Average_F1': 0.68, 'Average_Precision': 0.7, 'Average_Recall': 0.68}
{'(10_epoch)_(2e-6_LR)_(128_batch)': {'Accuracy': 0.7, 'Average_F1': 0.68, 'Average_Precision': 0.7, 'Average_Recall': 0.68}}


#### 15 epoch

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "MARBERT"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=128, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=15, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=15` reached.


##### Training Results

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))



  0%|          | 0/16 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       anger       0.68      0.74      0.71       290
        fear       0.92      0.89      0.91       219
         joy       0.66      0.49      0.57       241
        love       0.76      0.75      0.75       227
        none       0.67      0.84      0.74       316
     sadness       0.56      0.48      0.52       256
    surprise       0.43      0.44      0.44       167
    sympathy       0.85      0.83      0.84       219

    accuracy                           0.69      1935
   macro avg       0.69      0.68      0.68      1935
weighted avg       0.69      0.69      0.69      1935



In [ ]:
results={}
sum_f1=0
sum_precision=0
sum_recall=0
for i in lbl_enc.classes_:
  sum_f1 += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['f1-score']
  sum_precision += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['precision']
  sum_recall += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['recall']

results['Accuracy'] =round(classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)['accuracy'],2)
results['Average_F1'] =  round(sum_f1 /8,2)
results['Average_Precision'] =  round(sum_precision /8,2)
results['Average_Recall'] =  round(sum_recall /8,2)


Overall_Results['(15_epoch)_(2e-6_LR)_(128_batch)'] = results

print(results)
print(Overall_Results)

{'Accuracy': 0.69, 'Average_F1': 0.68, 'Average_Precision': 0.69, 'Average_Recall': 0.68}
{'(10_epoch)_(2e-6_LR)_(128_batch)': {'Accuracy': 0.7, 'Average_F1': 0.68, 'Average_Precision': 0.7, 'Average_Recall': 0.68}, '(15_epoch)_(2e-6_LR)_(128_batch)': {'Accuracy': 0.69, 'Average_F1': 0.68, 'Average_Precision': 0.69, 'Average_Recall': 0.68}}


#### 20 epoch 

In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "MARBERT"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=128, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=20, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

##### Training Results

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))



  0%|          | 0/16 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       anger       0.72      0.72      0.72       290
        fear       0.94      0.89      0.92       219
         joy       0.61      0.51      0.56       241
        love       0.80      0.76      0.78       227
        none       0.67      0.86      0.76       316
     sadness       0.57      0.53      0.55       256
    surprise       0.41      0.39      0.40       167
    sympathy       0.84      0.85      0.85       219

    accuracy                           0.70      1935
   macro avg       0.70      0.69      0.69      1935
weighted avg       0.70      0.70      0.70      1935



In [ ]:
results={}
sum_f1=0
sum_precision=0
sum_recall=0
for i in lbl_enc.classes_:
  sum_f1 += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['f1-score']
  sum_precision += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['precision']
  sum_recall += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['recall']

results['Accuracy'] =round(classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)['accuracy'],2)
results['Average_F1'] =  round(sum_f1 /8,2)
results['Average_Precision'] =  round(sum_precision /8,2)
results['Average_Recall'] =  round(sum_recall /8,2)


Overall_Results['(20_epoch)_(2e-6_LR)_(128_batch)'] = results

print(results)
print(Overall_Results)

{'Accuracy': 0.7, 'Average_F1': 0.69, 'Average_Precision': 0.7, 'Average_Recall': 0.69}
{'(10_epoch)_(2e-6_LR)_(128_batch)': {'Accuracy': 0.7, 'Average_F1': 0.68, 'Average_Precision': 0.7, 'Average_Recall': 0.68}, '(15_epoch)_(2e-6_LR)_(128_batch)': {'Accuracy': 0.69, 'Average_F1': 0.68, 'Average_Precision': 0.69, 'Average_Recall': 0.68}, '(20_epoch)_(2e-6_LR)_(128_batch)': {'Accuracy': 0.7, 'Average_F1': 0.69, 'Average_Precision': 0.7, 'Average_Recall': 0.69}}


#### 30 epoch


In [ ]:
# TODO: getting different models sizes results
MODEL_TYPE = "MARBERT"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=128, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=30, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

##### Training Results

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))



  0%|          | 0/16 [00:00<?, ?it/s]

              precision    recall  f1-score   support

       anger       0.70      0.70      0.70       290
        fear       0.94      0.89      0.92       219
         joy       0.66      0.49      0.56       241
        love       0.76      0.76      0.76       227
        none       0.69      0.84      0.76       316
     sadness       0.53      0.53      0.53       256
    surprise       0.45      0.47      0.46       167
    sympathy       0.85      0.82      0.83       219

    accuracy                           0.70      1935
   macro avg       0.70      0.69      0.69      1935
weighted avg       0.70      0.70      0.69      1935



In [ ]:
results={}
sum_f1=0
sum_precision=0
sum_recall=0
for i in lbl_enc.classes_:
  sum_f1 += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['f1-score']
  sum_precision += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['precision']
  sum_recall += classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)[i]['recall']

results['Accuracy'] =round(classification_report(real_values, preds, target_names=lbl_enc.classes_, output_dict=True)['accuracy'],2)
results['Average_F1'] =  round(sum_f1 /8,2)
results['Average_Precision'] =  round(sum_precision /8,2)
results['Average_Recall'] =  round(sum_recall /8,2)


Overall_Results['(30_epoch)_(2e-6_LR)_(128_batch)'] = results

print(results)
print(Overall_Results)

{'Accuracy': 0.7, 'Average_F1': 0.69, 'Average_Precision': 0.7, 'Average_Recall': 0.69}
{'(10_epoch)_(2e-6_LR)_(128_batch)': {'Accuracy': 0.7, 'Average_F1': 0.68, 'Average_Precision': 0.7, 'Average_Recall': 0.68}, '(15_epoch)_(2e-6_LR)_(128_batch)': {'Accuracy': 0.69, 'Average_F1': 0.68, 'Average_Precision': 0.69, 'Average_Recall': 0.68}, '(20_epoch)_(2e-6_LR)_(128_batch)': {'Accuracy': 0.7, 'Average_F1': 0.69, 'Average_Precision': 0.7, 'Average_Recall': 0.69}, '(30_epoch)_(2e-6_LR)_(128_batch)': {'Accuracy': 0.7, 'Average_F1': 0.69, 'Average_Precision': 0.7, 'Average_Recall': 0.69}}


In [ ]:
tuning_result = pd.DataFrame(Overall_Results)
tuning_result.columns=['10 epoch', '20 epoch', '15 epoch','30 epoch']

In [ ]:
tuning_result

,10 epoch,20 epoch,15 epoch,30 epoch
Accuracy,0.70,0.69,0.70,0.70
Average_F1,0.68,0.68,0.69,0.69
Average_Precision,0.70,0.69,0.70,0.70
Average_Recall,0.68,0.68,0.69,0.69


In [ ]:
styles = [dict(selector="caption", props=[("font-size", "120%"),
                                          ("font-weight", "bold")])]

tuning_result.style.set_caption('MarBERT With LR = 2e-6').set_table_styles(styles)

,10 epoch,20 epoch,15 epoch,30 epoch
Accuracy,0.700000,0.690000,0.700000,0.700000
Average_F1,0.680000,0.680000,0.690000,0.690000
Average_Precision,0.700000,0.690000,0.700000,0.700000
Average_Recall,0.680000,0.680000,0.690000,0.690000
